In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df = pd.read_csv('2006_2017_features.csv')

In [3]:
df.loc[:,'date'] = pd.to_datetime(df['date'])

In [4]:
df.set_index('date', inplace=True)

In [5]:
df['day'] = df.index.day.astype(int)

In [6]:
df.shape

(1827, 15)

In [7]:
df.reset_index(inplace=True)

In [8]:
df.tail()

,date,year,index,N1,N2,N3,N4,N5,N6,S1,color,zodiac,EvO,BS,month,day
1822,2017-12-21,2017,149,47,30,21,28,14,22,3,blue,goat,odd,small,12,21
1823,2017-12-24,2017,150,34,39,47,1,16,26,11,green,pig,odd,small,12,24
1824,2017-12-26,2017,151,37,2,7,30,16,28,12,red,dog,even,small,12,26
1825,2017-12-28,2017,152,46,15,40,44,17,23,11,green,pig,odd,small,12,28
1826,2017-12-30,2017,153,13,1,25,38,36,22,37,blue,rooster,odd,big,12,30


In [9]:
blue = []
red = []
green = []
for i in range(1,1828):
    blue.append(df['color'].iloc[:i].value_counts()['blue'])
    
for i in range(2,1828):
    red.append(df['color'].iloc[:i].value_counts()['red'])
    
for i in range(3,1828):
    green.append(df['color'].iloc[:i].value_counts()['green'])

blue = [0] + blue
red = [0,0] + red
green = [0,0,0] + green

df['red'] = red[0:1827]
df['green'] = green[0:1827]
df['blue'] = blue[0:1827]

In [10]:
df_clean = df[['year','color','red','green','blue','day','index']]

In [11]:
dict_color = {'red':0, 'green':1, 'blue':2}
color = df_clean['color'].map(dict_color)
df_clean.loc[:,'color'] = color

/Users/gznl/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [12]:
df_clean.head()

,year,color,red,green,blue,day,index
0,2006,2,0,0,0,3,1
1,2006,0,0,0,1,5,2
2,2006,1,1,0,1,8,3
3,2006,2,1,1,1,10,4
4,2006,0,1,1,2,12,5


In [13]:
holdout = df_clean[df_clean['year']==2017]
df_clean = df_clean[df_clean['year']!=2017]
df_clean = df_clean.drop('year', axis=1)

In [14]:
from sklearn.model_selection import train_test_split

In [15]:
X = df_clean.drop('color', axis=1)
y = df_clean['color']

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [17]:
import tensorflow.contrib.learn as learn
import tensorflow as tf

/Users/gznl/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [18]:
feature_columns = [tf.contrib.layers.real_valued_column("", dimension=5)]
clf = learn.DNNClassifier(hidden_units=[10,10,5], n_classes=3, feature_columns=feature_columns)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1c16e4a0f0>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/var/folders/q2/cdtm19053z38trwxwgrqzg0w0000gn/T/tmp4vmkeiga'}


In [19]:
clf.fit(X_train, y_train, steps=10, batch_size=20)

Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
Instructions for updating:
Please switch to tf.train.get_global_step
INFO:tensorflow:Cre

DNNClassifier(params={'head': <tensorflow.contrib.learn.python.learn.estimators.head._MultiClassHead object at 0x1c16e3cd68>, 'hidden_units': [10, 10, 5], 'feature_columns': (_RealValuedColumn(column_name='', dimension=5, default_value=None, dtype=tf.float32, normalizer=None),), 'optimizer': None, 'activation_fn': <function relu at 0x10b031840>, 'dropout': None, 'gradient_clip_norm': None, 'embedding_lr_multipliers': None, 'input_layer_min_slice_size': None})

In [20]:
pred_dnn = clf.predict(X_test)

Instructions for updating:
Please switch to predict_classes, or set `outputs` argument.
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
INFO:tensorflow:Restoring parameters from /var/folders/q2/cdtm19053z38trwxwgrqzg0w0000gn/T/tmp4vmkeiga/model.ckpt-10


In [21]:
from sklearn.metrics import confusion_matrix, classification_report

In [22]:
a = []
for i in pred_dnn:
    a.append(i)

In [23]:
print(confusion_matrix(y_test, a))
print('\n')
print(classification_report(y_test, a))

[[185   0   8]
 [159   0   7]
 [186   0   8]]


             precision    recall  f1-score   support

          0       0.35      0.96      0.51       193
          1       0.00      0.00      0.00       166
          2       0.35      0.04      0.07       194

avg / total       0.24      0.35      0.20       553



/Users/gznl/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
